In [1]:
#Get information about the Austin.aprx project file
import arcpy
import os
pathname = r"C:\GISprojects\Assignment8\ex8"
aprx = arcpy.mp.ArcGISProject(os.path.join(pathname,"Austin.aprx"))
print(aprx.defaultGeodatabase)

C:\GISprojects\Assignment8\ex8\Austin_Data.gdb


In [2]:
#Save a copy of the project
aprx.saveACopy(os.path.join(pathname,'Austin_Copy.aprx'))

When I ran the code and created a copy of the project, all items in the new project were the same as in the original. The file size of the copy was slightly smaller because the copy process removes temporary metadata, cached information, and editing history that may have been stored in the original project.

In [3]:
#Print a list of maps in the project 
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print(m.name)
    print(m.mapUnits)
del aprx

Downtown
Foot_US
County
Foot_US


The del aprx in the last line did not delete the project file. It removes the variable reference from memory in Python session.

In [6]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Region")[0]
m.name = "County"
del aprx

The map Region is renamed to County.

In [7]:
#List the layers in a map
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print("Map: " + m.name)
    lyrs = m.listLayers()
    for lyr in lyrs:
        print(lyr.name)
del aprx

Map: Downtown
Light Gray Reference
trees
parks
base
Light Gray Base
Map: County
facilities
hospitals
parks
Topographic


In [8]:
#Print if a layer is a basemap or a feature layer
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
lyrs = m.listLayers()
for lyr in lyrs:
    if lyr.isBasemapLayer:
        print(lyr.name + " is a basemap layer")
    if lyr.isFeatureLayer:
        print(lyr.name + " is a feature layer")
del aprx


Light Gray Reference is a basemap layer
trees is a feature layer
parks is a feature layer
base is a feature layer
Light Gray Base is a basemap layer


In [9]:
# Change the basemap of Downtown
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
m.addBasemap("Light Gray Canvas")

The Light Gray Canvas basemap is added to the project.

In [10]:
#Change the symbology of layers
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
green = {"RGB": [100, 175, 0, 100]}
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = green
    lyr.symbology = sym

The variable green is a Python dictionary containing RGBA values. The list represents the red, green, blue, and alpha (transparency) channels, producing green color.

In [11]:
#Change to brown color
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
brown = {"RGB": [150, 75, 0, 100]}   # changed to brown
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = brown
    lyr.symbology = sym


The parks layer changes its symbology to brown color.

In [12]:
#Add a layout to the project
m = aprx.listMaps("Downtown")[0]

lyt = aprx.createLayout( 11,8.5, 'INCH', 'New Layout with Rectangles')

def MakeRec_LL(llx, lly, w, h):
    xyRecList = [[llx, lly], [llx, lly+h], [llx+w,lly+h], [llx+w,lly], [llx,lly]]
    array = arcpy.Array([arcpy.Point(*coords) for coords in xyRecList])
    rec = arcpy.Polygon(array)
    return rec

mf = lyt.createMapFrame(MakeRec_LL(0.5,0.5,10,7.5), m, "New Map Frame")


The layout created is 11 inches wide by 8.5 inches high.The map frame is defined as 10 inches wide by 7.5 inches high, positioned 0.5 inches from the lower‑left corner of the layout. Creating layouts and map frames in Python offers automation, consistency, efficiency, and flexibility compared to manual creation.

In [13]:
#Create a north arrow
naStyle = aprx.listStyleItems('ArcGIS 2D', 'North_Arrow', 'Compass North 1')[0]
na = lyt.createMapSurroundElement(arcpy.Point(9.5,7.5), 'North_Arrow', mf,
                                      naStyle, "Compass North Arrow")
na.elementWidth = 0.5  

#Create a scale bar
sbName = 'Double Alternating Scale Bar 1 Metric'
sbStyle = aprx.listStyleItems('ArcGIS 2D', 'Scale_bar', sbName)[0]
sbEnv = MakeRec_LL(5.5, 0.1, 4, 0.5)
sb = lyt.createMapSurroundElement(sbEnv, 'Scale_bar', mf, sbStyle, 'New Scale Bar')


In [14]:
#Create a legend
legSi = aprx.listStyleItems('ArcGIS 2D', 'LEGEND', 'Legend 3' )[0]
leg = lyt.createMapSurroundElement(arcpy.Point(1,7), 'LEGEND', mf, legSi, 'New Legend Element')
leg.elementWidth = 3
leg.elementHeight = 3
leg.fittingStrategy = 'AdjustFontSize'
leg.columnCount = 1
leg.title = 'Downtown'


In [15]:
#Export the layout to a pdf file
lyt.exportToPDF(os.path.join(pathname, 'downtown.pdf'))

'C:\\GISprojects\\Assignment8\\ex8\\downtown.pdf'